# Exemplo 1


Instalar biliotecas necessárias

In [2]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.3 MB/s eta 0:00:00


In [3]:
from enum import auto, IntFlag # enumeração de objetos
from typing import Any # Definição de tipos de dados
from pydantic import (BaseModel,EmailStr,Field, SecretStr,ValidationError) # validador de dados

Criação de classe para definir os papeis disponíveis aos usuários

In [4]:
class Role(IntFlag): # definindo classe de papel de usuário
    Author = auto() # função auto(): defini posição de cada papel automaticamente, de maneira que os status não entrem em conclito.
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer # papel com funções combinadas

Classe com demais dados a serem registrados, como nome, email, senha e papel

In [5]:
class User(BaseModel): # Definindo classe de usuário
    name: str = Field(examples=["Arjan"]) # nome, cujo tipo de dado deve ser string
    email: EmailStr = Field( # email cujo o tipo deve ser string especializada para emails
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True, # campo não pode ser alterado posteriormente
    )
    password: SecretStr = Field( # senha, cujo tipo é string especializadas para senhas
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user") # papel de usuário, recebendo None como valor padrão

Função de validação para comparar estrutura dos dos dados a serem recebidos com a estrutura esperada pela classe User

In [9]:
def validate(data: dict[str, Any]) -> None: # função de validação
    try:
        user = User.model_validate(data) # retorna campos preenchidos caso não haja erros
        print(user)
    except ValidationError as e: # Em caso de erro, printa-los para identificar quais inconsistencias estão presentes
        print("User is invalid")
        for error in e.errors():
            print(error)

Definindo função que executará os testes de validação a partir de dados fictícios

In [26]:
def main() -> None: # função para teste de preenchimento de dados

    good_data = { # dados corretos estrturalmente
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }

    bad_data = {"email": "<bad data>", "password": "<bad data>"} # dados errados estruturalmente

    validate(good_data) # aplicando as funções de validação nos exemplos de input
    validate(bad_data)

Na execução, o primeiro caso de good_data atende corretamente às regras da classe User. Já no segundo, foram identificadas duas inconsistências: ausência do campo obrigatório "name" e formato de e-mail inválido por não conter o símbolo "@".

In [27]:
if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}
